
# **Abdullah Farooq - 2020023**

## Grammar

● program -> stmt_list

● stmt_list -> stmt stmt_list | ε

● stmt -> assignment_stmt | if_stmt | print_stmt

● assignment_stmt -> VARIABLE = expr ;

● if_stmt -> if ( expr ) { stmt_list } else { stmt_list } | if ( expr ) { stmt_list }

● print_stmt -> print ( expr ) ;

● expr -> term expr'

● expr' -> + term expr' | - term expr' | ε

● term -> factor term'

● term' -> * factor term' | ε

● factor -> NUMBER | VARIABLE | (expr)

● NUMBER -> [0-9]+

● VARIABLE -> [a-zA-Z]+


## Code

In [55]:
import re

class Parser:
    def __init__(self, input_string):
        # Tokenize the input string and initialize parser variables
        self.input_tokens = self.tokenize(input_string)
        self.current_token = None
        self.pos = -1
        self.advance()

    def tokenize(self, input_string):
        # Tokenize the input string using regular expressions
        # Tokens include identifiers, numbers, operators, and special characters
        return re.findall(r'//.*?$|/\*.*?\*/|\b\w+\b|[^\w\s]', input_string, re.MULTILINE|re.DOTALL)

    def advance(self):
        # Move to the next token in the input
        self.pos += 1
        if self.pos < len(self.input_tokens):
            self.current_token = self.input_tokens[self.pos]
        else:
            self.current_token = None

    def error(self):
        # Raise a syntax error
        raise Exception("Syntax Error")

    def expr(self):
        # Parse an expression following the grammar rules
        result = self.term()
        while self.current_token in ('+', '-'):
            op = self.current_token
            self.advance()
            result = (op, result, self.term())
        return result

    def term(self):
        # Parse a term following the grammar rules
        result = self.factor()
        while self.current_token in ('*', '/'):
            op = self.current_token
            self.advance()
            result = (op, result, self.factor())
        return result

    def factor(self):
        # Parse a factor following the grammar rules
        if self.current_token.isdigit():
            return self.number()
        elif self.current_token.isalpha():
            return self.variable()
        elif self.current_token == '(':
            self.advance()
            result = self.expr()
            if self.current_token != ')':
                self.error()
            self.advance()
            return result
        else:
            self.error()

    def number(self):
        # Parse a number token
        result = ''
        while self.current_token is not None and self.current_token.isdigit():
            result += self.current_token
            self.advance()
        return int(result)

    def variable(self):
        # Parse a variable token
        result = ''
        while self.current_token is not None and self.current_token.isalpha():
            result += self.current_token
            self.advance()
        return result

    def assignment_stmt(self):
        # Parse an assignment statement
        variable = self.variable()
        if self.current_token != '=':
            self.error()
        self.advance()
        expr_result = self.expr()
        if self.current_token != ';':
            self.error()
        self.advance()
        return ('=', variable, expr_result)

    def if_stmt(self):
        # Parse an if statement
        if self.current_token != 'if':
            self.error()
        self.advance()
        if self.current_token != '(':
            self.error()
        self.advance()
        condition = self.expr()
        if self.current_token not in ('<', '>', '<=', '>=', '=='):
            self.error()
        op = self.current_token
        self.advance()
        second_condition = self.expr()
        if self.current_token != ')':
            self.error()
        self.advance()
        if_block = self.stmt_list()
        else_block = []
        if self.current_token == 'else':
            self.advance()
            else_block = self.stmt_list()
        return ('if', condition, op, second_condition, if_block, else_block)

    def print_stmt(self):
        # Parse a print statement
        if self.current_token != 'print':
            self.error()
        self.advance()
        if self.current_token != '(':
            self.error()
        self.advance()
        expr_result = self.expr()
        if self.current_token != ')':
            self.error()
        self.advance()
        if self.current_token != ';':
            self.error()
        self.advance()
        return ('print', expr_result)

    def stmt(self):
        # Determine the type of statement and call the corresponding parsing method
        if self.current_token == 'if':
            return self.if_stmt()
        elif self.current_token == 'print':
            return self.print_stmt()
        else:
            return self.assignment_stmt()

    def stmt_list(self):
        # Parse a list of statements
        statements = []
        while self.current_token is not None and self.current_token != '}':
            statement = self.stmt()
            statements.append(statement)
        return statements

    def parse(self):
        # Entry point for parsing, starts with stmt_list
        return self.stmt_list()

def test_parser(expression):
    # Test the parser with an input expression
    parser = Parser(expression)
    try:
        result = parser.parse()
        print("Expression:", expression)
        print("Parsed Result:", result)
    except Exception as e:
        print("Expression:", expression)
        print("Error:", e)
    print()

# Test the parser with various input expressions
test_parser("a = 5;")
test_parser("x = 5; print(x / 5);")
test_parser("print(4 + 7 + 5);")
test_parser("if (x > 2) print(x);")


test_parser("a \ 5") # Invalid syntax
test_parser("3 * [ b - 2]") # Invalid syntax
test_parser("x / {y + 7}")  # Invalid syntax
test_parser("a = 5") # Invalid syntax


test_parser("if (x > 2) print(x); else print(z); ") # Valid Syntax but not parsing accurately

Expression: a = 5;
Parsed Result: [('=', 'a', 5)]

Expression: x = 5; print(x / 5);
Parsed Result: [('=', 'x', 5), ('print', ('/', 'x', 5))]

Expression: print(4 + 7 + 5);
Parsed Result: [('print', ('+', ('+', 4, 7), 5))]

Expression: if (x > 2) print(x);
Parsed Result: [('if', 'x', '>', 2, [('print', 'x')], [])]

Expression: a \ 5
Error: Syntax Error

Expression: 3 * [ b - 2]
Error: Syntax Error

Expression: x / {y + 7}
Error: Syntax Error

Expression: a = 5
Error: Syntax Error

Expression: if (x > 2) print(x); else print(z); 
Error: Syntax Error

